# Installing necessary dependencies

In [ ]:
!pip install transformers datasets accelerate

# Tokenizer Initialization

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Loading Dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("text", data_files={"train": "Feliz.txt"})


Generating train split: 0 examples [00:00, ? examples/s]

# Tokenizing Data

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()  # this is the fix!
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

# Setting Training Arguements

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-Feliz-model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=5,
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
)


# Training the model

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,2.405500
100,0.082200
150,0.074400
200,0.073500
250,0.055200
300,0.053900
350,0.051900
400,0.058100
450,0.042900
500,0.042900


TrainOutput(global_step=612, training_loss=0.24803138548642203, metrics={'train_runtime': 978.6308, 'train_samples_per_second': 3.118, 'train_steps_per_second': 0.625, 'total_flos': 2833465819004928.0, 'train_loss': 0.24803138548642203, 'epoch': 3.0})

In [ ]:
trainer.save_model("./gpt2-character-trained")
tokenizer.save_pretrained("./gpt2-character-trained")

('./gpt2-character-trained/tokenizer_config.json',
 './gpt2-character-trained/special_tokens_map.json',
 './gpt2-character-trained/vocab.json',
 './gpt2-character-trained/merges.txt',
 './gpt2-character-trained/added_tokens.json')

# Inference Time

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
model = GPT2LMHeadModel.from_pretrained("./gpt2-character-trained")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-character-trained")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def query_model_simple(prompt, max_new_tokens=100):
    encoded = tokenizer(prompt, return_tensors="pt", truncation=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()
prompt = "Where is your GirlFriend toppo??"
print(query_model_simple(prompt))

😳
